In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [ ]:
item_names = sorted(df['type_name'].unique())

In [ ]:
dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Search item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)

In [ ]:
ma_slider = widgets.IntSlider(
    value=7,
    min=1,
    max=30,
    step=1,
    description='MA Window',
    continuous_update=False
)

In [ ]:
days_slider = widgets.IntSlider(
    value=365,
    min=1,
    max=len(df['date'].unique()),
    step=1,
    description='Days Shown',
    continuous_update=False
)

In [33]:
donchian_slider = widgets.IntSlider(
    value=20,
    min=5,
    max=90,
    step=1,
    description='Donchian Window',
    continuous_update=False
)

In [ ]:
output = widgets.Output()

In [ ]:
def plot_trend(type_name, ma_window, days_shown, donchian_window):
    with output:
        clear_output(wait=True)
        item_df = df[df['type_name'] == type_name].sort_values('date')
        if item_df.empty:
            print("No data for selected item.")
            return
        item_df = item_df.iloc[-days_shown:]

        # Calculate Donchian Channel
        donchian_high = item_df['highest'].rolling(window=donchian_window).max()
        donchian_low = item_df['lowest'].rolling(window=donchian_window).min()

        fig, ax1 = plt.subplots(figsize=(10, 5))

        # Donchian Channel fill
        ax1.fill_between(item_df['date'], donchian_low, donchian_high, color='cyan', alpha=0.1, label=f'Donchian {donchian_window}')

        # Daily average and MA
        ax1.plot(item_df['date'], item_df['average'], alpha=0.4, label='Daily Average', color='blue')
        ax1.plot(item_df['date'], item_df['average'].rolling(window=ma_window).mean(), color='red', label=f'{ma_window}-Day MA')

        # Donchian Channel upper/lower
        ax1.plot(item_df['date'], donchian_high, color='cyan', linestyle='--', label='Donchian High')
        ax1.plot(item_df['date'], donchian_low, color='deepskyblue', linestyle='--', label='Donchian Low')

        ax1.set_xlabel('Date')
        ax1.set_ylabel('Average Price (ISK)', color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')

        # Volume bars
        ax2 = ax1.twinx()
        ax2.bar(item_df['date'], item_df['volume'], width=1.0, color='gray', alpha=0.3, label='Daily Volume')
        ax2.set_ylabel('Volume', color='gray')
        ax2.tick_params(axis='y', labelcolor='gray')

        # Legends
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines + lines2, labels + labels2, loc='upper left')

        plt.title(f'{type_name} Price, Donchian Channel, & Volume\nLast {days_shown} Days')
        plt.tight_layout()
        plt.show()

interactive_plot = widgets.interactive(
    plot_trend,
    type_name=dropdown,
    ma_window=ma_slider,
    days_shown=days_slider,
    donchian_window=donchian_slider
)

display(interactive_plot, output)




ValueError: cannot find widget or abbreviation for argument: 'donchian_window'